In [17]:
import numpy as np 

import tensorflow as tf
from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras import layers, models
import matplotlib.pyplot as plt


In [19]:
data_dir = r"C:\Users\patid\Downloads\Driver drowsiness dataset\data\train"

train_ds = image_dataset_from_directory(
    data_dir,
    validation_split=0.2,
    subset="training",
    seed=42,
    image_size=(224, 224),
    batch_size=32
)

val_ds = image_dataset_from_directory(
    data_dir,
    validation_split=0.2,
    subset="validation",
    seed=42,
    image_size=(224, 224),
    batch_size=32
)


Found 50937 files belonging to 2 classes.
Using 40750 files for training.
Found 50937 files belonging to 2 classes.
Using 10187 files for validation.


In [20]:
# ⚙️ Preprocess dataset for MobileNetV2 (memory-friendly version)
train_ds = train_ds.map(lambda x, y: (preprocess_input(x), y))
val_ds = val_ds.map(lambda x, y: (preprocess_input(x), y))


In [21]:
base_model = MobileNetV2(
    weights='imagenet',
    include_top=False,
    input_shape=(224, 224, 3)
)
base_model.trainable = False  # freeze pretrained layers initially

model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dropout(0.3),
    layers.Dense(1, activation='sigmoid')  # binary classification
])

model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
    loss='binary_crossentropy',
    metrics=['accuracy']
)

model.summary()


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ mobilenetv2_1.00_224            │ (None, 7, 7, 1280)     │     2,257,984 │
│ (Functional)                    │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 1280)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 1280)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 1)              │         1,281 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,259,265 (8.62 MB)

 Trainable params: 1,281 (5.00 KB)

 Non-trainable params: 2,257,984 (8.61 MB)

In [22]:
#  Define callbacks properly
early_stop = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',
    patience=5,
    restore_best_weights=True
)

reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.5,
    patience=2,
    verbose=1
)

checkpoint = tf.keras.callbacks.ModelCheckpoint(
    filepath='mobilenet_model.h5',
    monitor='val_loss',
    save_best_only=True,
    verbose=1
)

#  Put all callbacks in a list
callbacks = [early_stop, reduce_lr, checkpoint]




In [ ]:


history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=20,
    callbacks=callbacks
)


Epoch 1/20


In [8]:
base_model.trainable = True
for layer in base_model.layers[:-30]:  # freeze all but last 30 layers
    layer.trainable = False

model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5),
    loss='binary_crossentropy',
    metrics=['accuracy']
)

fine_tune_history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=10,
    callbacks=callbacks
)


Epoch 1/10
1274/1274 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.9299 - loss: 0.1866
Epoch 1: val_loss improved from 0.08447 to 0.08319, saving model to mobilenet_model.h5


1274/1274 ━━━━━━━━━━━━━━━━━━━━ 3436s 3s/step - accuracy: 0.9519 - loss: 0.1317 - val_accuracy: 0.9708 - val_loss: 0.0832 - learning_rate: 1.0000e-05
Epoch 2/10
1274/1274 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.9734 - loss: 0.0763
Epoch 2: val_loss improved from 0.08319 to 0.04729, saving model to mobilenet_model.h5


1274/1274 ━━━━━━━━━━━━━━━━━━━━ 2778s 2s/step - accuracy: 0.9756 - loss: 0.0693 - val_accuracy: 0.9835 - val_loss: 0.0473 - learning_rate: 1.0000e-05
Epoch 3/10
1274/1274 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.9796 - loss: 0.0553
Epoch 3: val_loss improved from 0.04729 to 0.04403, saving model to mobilenet_model.h5


1274/1274 ━━━━━━━━━━━━━━━━━━━━ 2693s 2s/step - accuracy: 0.9812 - loss: 0.0509 - val_accuracy: 0.9841 - val_loss: 0.0440 - learning_rate: 1.0000e-05
Epoch 4/10
1274/1274 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.9848 - loss: 0.0442
Epoch 4: val_loss improved from 0.04403 to 0.03812, saving model to mobilenet_model.h5


1274/1274 ━━━━━━━━━━━━━━━━━━━━ 2509s 2s/step - accuracy: 0.9864 - loss: 0.0395 - val_accuracy: 0.9866 - val_loss: 0.0381 - learning_rate: 1.0000e-05
Epoch 5/10
1274/1274 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.9891 - loss: 0.0310
Epoch 5: val_loss did not improve from 0.03812
1274/1274 ━━━━━━━━━━━━━━━━━━━━ 2322s 2s/step - accuracy: 0.9893 - loss: 0.0301 - val_accuracy: 0.9864 - val_loss: 0.0388 - learning_rate: 1.0000e-05
Epoch 6/10
1274/1274 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.9905 - loss: 0.0277
Epoch 6: val_loss improved from 0.03812 to 0.03795, saving model to mobilenet_model.h5


1274/1274 ━━━━━━━━━━━━━━━━━━━━ 2490s 2s/step - accuracy: 0.9914 - loss: 0.0244 - val_accuracy: 0.9864 - val_loss: 0.0379 - learning_rate: 1.0000e-05
Epoch 7/10
1274/1274 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.9915 - loss: 0.0227
Epoch 7: val_loss did not improve from 0.03795
1274/1274 ━━━━━━━━━━━━━━━━━━━━ 2447s 2s/step - accuracy: 0.9924 - loss: 0.0209 - val_accuracy: 0.9870 - val_loss: 0.0381 - learning_rate: 1.0000e-05
Epoch 8/10
1274/1274 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.9933 - loss: 0.0180
Epoch 8: ReduceLROnPlateau reducing learning rate to 4.999999873689376e-06.

Epoch 8: val_loss did not improve from 0.03795
1274/1274 ━━━━━━━━━━━━━━━━━━━━ 2466s 2s/step - accuracy: 0.9941 - loss: 0.0168 - val_accuracy: 0.9869 - val_loss: 0.0395 - learning_rate: 1.0000e-05
Epoch 9/10
1274/1274 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.9954 - loss: 0.0131
Epoch 9: val_loss did not improve from 0.03795
1274/1274 ━━━━━━━━━━━━━━━━━━━━ 2439s 2s/step - accuracy: 0.9960 - loss: 0.

In [10]:
model.save("mobilenet_model_best.h5")
print("✅ Model saved successfully at mobilenet_model.h5")


✅ Model saved successfully at mobilenet_model.h5


In [28]:
# import pickle

# # Save history
# with open('training_history.pkl', 'wb') as f:
#     pickle.dump(history.history, f)


In [29]:
# plt.figure(figsize=(10,4))
# plt.subplot(1,2,1)
# plt.plot(history.history['accuracy'], label='Train Acc')
# plt.plot(history.history['val_accuracy'], label='Val Acc')
# plt.legend()
# plt.title("Model Accuracy")

# plt.subplot(1,2,2)
# plt.plot(history.history['loss'], label='Train Loss')
# plt.plot(history.history['val_loss'], label='Val Loss')
# plt.legend()
# plt.title("Model Loss")

# plt.show()


In [24]:
# Load trained model
model = tf.keras.models.load_model("mobilenet_model.h5")  

In [25]:
test_dir = r"C:\Users\patid\Downloads\Driver drowsiness dataset\data\test_small"

In [26]:
image_size = 224
batch_size = 32
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# Load test dataset as a tf.data.Dataset
test_ds = image_dataset_from_directory(
    test_dir,
    image_size=(image_size, image_size),
    batch_size=batch_size,
    shuffle=False
)

# Preprocess test dataset for MobileNetV2
test_ds = test_ds.map(lambda x, y: (preprocess_input(x), y))

# Get true labels from test_ds
true_labels = []
for _, labels in test_ds:
    true_labels.extend(labels.numpy())
true_labels = np.array(true_labels)

# Predict probabilities
predictions = model.predict(test_ds)
predicted_labels = (predictions > 0.5).astype(int).flatten()

# Accuracy, classification report, confusion matrix

acc = accuracy_score(true_labels, predicted_labels)
print(f"Accuracy: {acc:.4f}")

print("Classification Report:")
print(classification_report(true_labels, predicted_labels, target_names=class_names))

print("Confusion Matrix:")
print(confusion_matrix(true_labels, predicted_labels))

Found 2000 files belonging to 2 classes.
63/63 ━━━━━━━━━━━━━━━━━━━━ 98s 1s/step
Accuracy: 0.9890
Classification Report:
              precision    recall  f1-score   support

       awake       0.98      0.99      0.99      1000
      sleepy       0.99      0.98      0.99      1000

    accuracy                           0.99      2000
   macro avg       0.99      0.99      0.99      2000
weighted avg       0.99      0.99      0.99      2000

Confusion Matrix:
[[994   6]
 [ 16 984]]
